# Taxi data

Visualization of thier locus

## Import Module

In [1]:
# データ加工用モジュール
import  sys
import gc
import math
import copy

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# 地図用モジュール
import folium
from folium import plugins

print('folium version: \t\t', folium.__version__)

# Latitude列は小数第13位までの値を保持している。
# defaultでも表示されないだけで同値を保持しているが、見やすくするために指定。
pd.options.display.precision = 13
# default = 6
print('小数点以下表示桁数: \t', pd.options.display.precision)

folium version: 		 0.10.1
小数点以下表示桁数: 	 13


### データ加工ひとまとめ

In [2]:
# ----- データ読込部 --------------------------------------------------
# 列名指定
df_colnm = ('DriverNumber', 'StatusTime', 'CompanyID', 'Latitude', 
                   'Longitude', 'VehicleStatus', 'start', 'end')
# 列順は以下のようにする
# 'CompanyID', 'DriverNumber', 'StatusTime', 'Latitude', 
# 'Longitude', 'VehicleStatus', 'start', 'end'

# dtypeは辞書型で指定する
df_dtype = {'CompanyID': 'category', 'DriverNumber': 'category', 'Latitude': 'float64', 
                   'Longitude': 'float64', 'start': 'category', 'end': 'category'}

# 実際にデータ読込
df = pd.read_csv('taxi/company2_乗降フラグ付きデータ改訂版.csv',
                 header = None,
                 names = df_colnm,
                 usecols = [5, 1, 2, 8, 9, 13, 18, 19],
                 dtype = df_dtype,
                 nrows = 407680,
                 skiprows = 1,
                 encoding = 'SHIFT-JIS')

del df_colnm, df_dtype
gc.collect()
# ----- データ読込部 --------------------------------------------------

# データの絞り込み ----------
df = df.query('VehicleStatus == "賃走"')
df.drop('VehicleStatus', axis = 1)

# startフラグとendフラグの数が同値になっているかをチェック。 ----------
if (df['start'] == '1').sum() == (df['end'] == '1').sum():
    print('\'start\'の数と\'end\'の数：OK')
else:
    # startとendが同値でなければエラーとしてプログラム自体をbreakする。
    print('ERROR：\'start\'の数と\'end\'の数が等しくありません。 ')
    sys.exit()

# 外れ値を飛ばすために'start'と'end'列のdtype'category'を8bitの整数値のdtypeに変換
df['start'] = df['start'].astype('int8')
df['end'] = df['end'].astype('int8')
# 1回のみ'賃走'が検出された1経路は外れ値として飛ばす
df = df.query('not(start == 1 & end == 1)')
# メモリ確保のために'start'と'end'列のdtypeを'category'に戻す
df['start'] = df['start'].astype('category')
df['end'] = df['end'].astype('category')

# map出力用データ加工 ----------
df['StatusTime'] = df['StatusTime'].str[:10].str.cat(df['StatusTime'].str[-8:], sep = 'T')

# 列順変更 ----------
df = df.loc[:, ['CompanyID', 'DriverNumber', 'StatusTime', 'Latitude', 
                     'Longitude', 'VehicleStatus', 'start', 'end']]

# DriverNumber毎にColorCodeを割り当てたDataFrameの作成 ---------- ---------- ----------
# DriverNumberのuniqueなリストを作成 ----------
df_drivers = list(set(df['DriverNumber']))
print('ドライバー数: ', len(df_drivers))
# カラーコードのリスト作成
df_colors = plt.rcParams['axes.prop_cycle'].by_key()['color']
# ColorCodeのリストを参照用にdeep copyする
# Pythonは参照渡し。リスト型はmutableなので値渡しでcopy
df_colors_copy = copy.deepcopy(df_colors)
# ColorCodeのリストをDriver数によって連結させる
for i in range(math.floor((len(df_drivers) - 1) / 10)):
    df_colors.extend(df_colors_copy)
del df_colors_copy
gc.collect()

# DriverColorの空dataframeを作成
df_DriverColor = pd.DataFrame(columns = ['ColorCode'] )
# 1DriverNumberに対して1ColorCodeを割り当てる ----------
for i in range(len(df_drivers)):
    df_DriverColor = df_DriverColor.append(
        pd.Series(df_colors[i], index = df_DriverColor.columns), 
        ignore_index = True)
# 作成したColorCodeのDataFrameをDriverNumberのリストと
# concatenateするためにDriverNumberのリストをpandas.DataFrameに
df_drivers = pd.DataFrame(df_drivers)
# DriverNumberとColorCodeをconcatenate
df_DriverColor = pd.concat([df_drivers, df_DriverColor], axis = 1)
# 列名のrename
df_DriverColor.rename(columns = {0: 'DriverNumber'}, inplace = True)
del df_drivers, df_colors
gc.collect()
# DriverNumber毎にColorCodeを割り当てたDataFrameの作成 ---------- ---------- ----------

# 元のDataFrameとDriverNumber毎にColorCodeを割り当てたDataFrameをleft_join ----------
df = pd.merge(df, df_DriverColor, on = 'DriverNumber', how = 'left')
del df_DriverColor
gc.collect()

# 先頭から'1'が来るまでの行をグループ化 ---------- ---------- ---------- ---------- ----------
grouped_df = df.groupby((df['start'] == 1).cumsum())
# group化したdataframeを作成しているため、元のdataframeのメモリ解放
del df
gc.collect()

'start'の数と'end'の数：OK
ドライバー数:  1


0

In [3]:
# メモリ使用 可視化
print("{}{: >25}{}{: >10}{}".format('|','Variable Name','|','Memory','|'))
print(" ------------------------------------ ")
for var_name in dir():
    if not var_name.startswith("_"):
        print("{}{: >25}{}{: >10}{}".format('|',var_name,'|',sys.getsizeof(eval(var_name)),'|'))

|            Variable Name|    Memory|
 ------------------------------------ 
|                       In|        96|
|                      Out|       240|
|                     copy|        80|
|                     exit|        56|
|                   folium|        80|
|                       gc|        80|
|              get_ipython|        64|
|               grouped_df|        56|
|                        i|        24|
|                     math|        80|
|                       np|        80|
|                       pd|        80|
|                      plt|        80|
|                  plugins|        80|
|                     quit|        56|
|                      sys|        80|


# Make the map

In [4]:
# mapオブジェクトを定義
trajMap = folium.Map(
    # 新しい地図を作るときの中央にくる緯度と経度を入れる
    location = [35.6866158 , 139.73857073333332],
    # はじめに表示するときの縮尺を設定する
    zoom_start = 12
)

# linesを繰り返し代入する空のリストを定義
# for文の外で定義しなければならない
lines = []

# ----------
# start to end毎にlinesを作成
for j in range(1, len(grouped_df) + 1):
    # 日時データのみ抽出
    df_date = grouped_df.get_group(j)['StatusTime']

    # 緯度・経度データのみ抽出
    df_lat = grouped_df.get_group(j)['Latitude']
    df_lon = grouped_df.get_group(j)['Longitude']

    # 色データのみ抽出
    df_color = grouped_df.get_group(j)['ColorCode']

    # data frameの行数より1少ない値分繰り返す
    for i in range(len(grouped_df.get_group(j)) - 1):
        # 緯度・経度を'lines'に入れる(linesは辞書型リスト)
        line = [
            {
                # 座標点 *** 経度, 緯度の順番でリストに入れなければならない点に注意! ***
                'coordinates':[
                    [df_lon.iloc[i], df_lat.iloc[i]], 
                    [df_lon.iloc[i + 1], df_lat.iloc[i + 1]]
                ],
                # 日時
                'dates':[
                    df_date.iloc[i],
                    df_date.iloc[i + 1]
                ],
                # 線色
                'color': df_color.iloc[i],
                # 線の太さ
                'weight': 5
            }
        ] # ----- lines END -----
        # from IPython.core.debugger import Pdb; Pdb().set_trace()
        lines.extend(line)
# ----------


# featuresに入れる際に参照するlineオブジェクトを初期化
line = []
# featureを繰り返し代入するfeaturesリストを定義
# 
features = []
for line in lines:
    feature = [
        {
            'type': 'Feature',
            'geometry': {
                'type': 'LineString',
                'coordinates':line['coordinates'],
            },
            'properties': {
                'times': line['dates'],
                'style': {
                    'color': line['color'],
                    'weight': line['weight']
                }
            }
        }
    ]
    features.extend(feature)
# featuresにlinesで必要な要素を全て入れたので用無し。
# 他にもメモリを確保している変数のメモリを解放する。
del lines, line, feature, df_date, df_lat, df_lon, df_color, i, j
gc.collect()

0

In [ ]:
plugins.TimestampedGeoJson(
    {
        'type': 'FeatureCollection',
        'features': features,
    }, 
    period = 'PT1M', 
    add_last_point = False).add_to(trajMap)

# trajMap.save('trajMap.html')
trajMap

In [ ]:
trajMap.save('trajMap.html')